In [55]:
import json

local_config_file = "./local.settings.json"

with open(local_config_file) as f:
    config = json.load(f)

api_path = config["DATA"]["API"]["Path"]
api_key = config["DATA"]["API"]["Key"]
user_id = config["DATA"]["API"]["UserID"]
organization_id = config["DATA"]["API"]["OrganizationID"]
user_name = config["DATA"]["API"]["UserName"]
process_files = config["DATA"]["API"]["Methods"]["ProcessFiles"]

local_files_folder = config["Local"]["FilesFolder"]

#print(local_files_folder)

In [56]:
import os

files_to_upload = []

for root, dirs, files in os.walk(local_files_folder):
    for file in files:
        if file.endswith(".txt"):
            files_to_upload.append(os.path.join(root, file))
            #print(os.path.join(root, file))

In [57]:
from typing import List
    
def get_mime_type(file_path:str):
    mime_type = "application/octet-stream"

    if file_path.endswith(".pdf"):
        mime_type = "application/pdf"
    elif file_path.endswith(".docx"):
        mime_type = "application/vnd.openxmlformats-officedocument.wordprocessingml.document"
    elif file_path.endswith(".doc"):
        mime_type = "application/msword"
    elif file_path.endswith(".txt"):
        mime_type = "text/plain"

    return mime_type

class FileProcessingInputData:
    UserId:str
    OrganizationId:str
    UserName:str
    FileId:str
    ScoreMarginalDocuments:bool
    Tags:List[str]
    Properties:dict

def get_file_data(file_path:str):
    file_data = FileProcessingInputData()
    file_data.UserId = user_id
    file_data.OrganizationId = organization_id
    file_data.UserName = user_name
    file_data.FileId = os.path.basename(file_path)
    file_data.ScoreMarginalDocuments = True

    return file_data

In [58]:
import requests
from io import StringIO

api_process_files_path = f'{api_path}/{process_files}'

def upload_document(file_data:FileProcessingInputData, file_path:str):
    
    
    
    url = api_process_files_path
    headers = {"Data-Subscription-Key": api_key}

    data_json = StringIO(json.dumps(file_data, default=vars))
    file = open(file_path, "rb").read()
    files = {"json": (None, data_json, "application/json; charset=UTF-8"), file_data.FileId: (file_data.FileId, file, get_mime_type(file_path))}

    response = requests.post(url, headers=headers, files=files)
    print(str(response.status_code) + " " + response.reason + " " + response.text)


In [59]:
for file_path in files_to_upload:
    file_data = get_file_data(file_path)
    upload_document(file_data, file_path)

202 Accepted PROCESSING
202 Accepted PROCESSING
202 Accepted PROCESSING
202 Accepted PROCESSING
202 Accepted PROCESSING
202 Accepted PROCESSING
